In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import wget

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page

<Response [200]>

## Parse the url

In [6]:
soup = BeautifulSoup(page.text,'html.parser')

In [7]:
#print(soup.prettify())

## Find the html table from the parser

In [8]:
table=soup.find_all('table')[0]

In [9]:
df = pd.DataFrame(columns=range(2))

## Update the table with row and column value 

In [10]:
row_index=0
for row in table.find_all('tr'):
    col_index=0
    columns = row.find_all('td')
    for col in columns:
        df.loc[row_index,col_index] = col.get_text().replace('\n','')
        col_index += 1
    row_index += 1

In [11]:
df.reset_index(drop=True)
df.columns=['PostalCode','Borough','Neighborhood']

## Remove Borough with "Not assigned"

In [12]:
df.drop(df[df['Borough'] == 'Not assigned'].index,axis=0,inplace=True)
df=df.reset_index(drop=True)

## Replace / with ,

In [13]:
df['Neighborhood']=df['Neighborhood'].str.replace('/',',')

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## Checking for nan value in Neighborhood column

In [15]:
df.isnull().any()

PostalCode      False
Borough         False
Neighborhood    False
dtype: bool

In [16]:
df.shape

(103, 3)

## Load the coordinates file

In [17]:
wget.download('http://cocl.us/Geospatial_data','/Users/skdb/Desktop/Projects/Coursera_Capstone')

'/Users/skdb/Desktop/Projects/Coursera_Capstone/Geospatial_Coordinates (1).csv'

In [18]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

## Merge two dataframe

In [20]:
toronto_merge = df
toronto_merge = toronto_merge.merge(coordinates_df,on='PostalCode')
toronto_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


## Get totoal Borough and neighborhood

In [21]:
print("The dataframe has {} Borough and {} Neighbothood".format(
                           len(toronto_merge['Borough'].unique()),
                           toronto_merge.shape[0]))

The dataframe has 10 Borough and 103 Neighbothood


In [22]:
toronto_merge['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## Check Borough column contains "Toronto"

In [23]:
#toronto_only = toronto_merge[toronto_merge['Borough'].str.contains('Toronto') ].reset_index(drop=True)
#toronto_all = toronto_merge

In [24]:
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import requests

from pandas.io.json import json_normalize

In [25]:
address = "Toronto"

geolocator = Nominatim(user_agent='totonto_explorer')
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude
print("Toronto latitude {} and longitude {}".format(latitude,longitude))

Toronto latitude 43.6534817 and longitude -79.3839347


In [26]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)
map_toronto

##  Create a map to superimpose neighborhood of toronto

In [27]:
toronto_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [28]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat, log, borough, neighbor in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Borough'],toronto_merge['Neighborhood']):
    label = '{}, {}'.format(neighbor,borough)
    folium.Popup(label,parse_html=True)
    folium.CircleMarker(
           [lat,log],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_capacity=0.7,
            parse_html = False).add_to(map_toronto)

map_toronto

## Define Foursquare credential and version

In [29]:
CLIENT_ID = 'NGCRL0SWNNL3PXIGLXOAR1XKKPCYD1ORE5QZJCBPKV2ZBFR1' # your Foursquare ID
CLIENT_SECRET = 'XLAS2RXZOZMCRNWPTS1D5PFLTGVBDGF03WPQD2MNIFZQ4J3R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NGCRL0SWNNL3PXIGLXOAR1XKKPCYD1ORE5QZJCBPKV2ZBFR1
CLIENT_SECRET:XLAS2RXZOZMCRNWPTS1D5PFLTGVBDGF03WPQD2MNIFZQ4J3R


## Let's explore the first neighborhood in our dataframe.

In [30]:
toronto_merge.loc[0,'Neighborhood']

'Parkwoods'

## Get the longitude and latitude

In [31]:
neighborhood_latitude = toronto_merge.loc[0,'Latitude']
neighborhood_longitude = toronto_merge.loc[0,'Longitude']
neighborhood_name = toronto_merge.loc[0,'Neighborhood']
print("{} has latitude {} and longitude {}".format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Parkwoods has latitude 43.7532586 and longitude -79.3296565


### Get the url for exploring 100 venue 

In [32]:
radius=500
LIMIT=100
url  ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,
                                                                                                                            neighborhood_latitude,
                                                                                                                            neighborhood_longitude,
                                                                                                                            radius,
                                                                                                                            LIMIT )
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NGCRL0SWNNL3PXIGLXOAR1XKKPCYD1ORE5QZJCBPKV2ZBFR1&client_secret=XLAS2RXZOZMCRNWPTS1D5PFLTGVBDGF03WPQD2MNIFZQ4J3R&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [33]:
results = requests.get(url).json()

In [34]:
venues = results['response']['groups'][0]['items']

In [35]:
nearby_venues = json_normalize(venues)
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8d9dcdd5fbbbb6b3003c7b-0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4e8d9dcdd5fbbbb6b3003c7b,Toronto,CA,Toronto,Canada,245,"[Toronto, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197604605557...",43.751976,-79.332140,ON,Brookbanks Park,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb11e2075ebb60cd1c4caad-1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",4cb11e2075ebb60cd1c4caad,29 Valley Woods Road,CA,Toronto,Canada,312,"[29 Valley Woods Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197441585782...",43.751974,-79.333114,ON,Variety Store,0,[]


In [36]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [39]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

In [40]:
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [41]:
nearby_venues.shape

(2, 4)

## Explore the neighborhood in Toronto

In [42]:
def getNearbyVenues(name, latitude, longitude, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(name, latitude, longitude):
        print(name)
        
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
               CLIENT_ID,
               CLIENT_SECRET,
               VERSION,
               lat,
               lng,
               radius,
               LIMIT)
        #print(url)
        #make the request
        results = requests.get(url)
        #print(results)
        results = results.json()['response']['groups'][0]['items']
        #print(results)
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])            
    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
toronto_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [45]:
toronto_venues = getNearbyVenues(toronto_merge['Neighborhood'],
                                 toronto_merge['Latitude'],
                                toronto_merge['Longitude'])

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District , Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
D

In [46]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


## Let's check how many venues were returned for each neighborhood

In [47]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood , Long Branch",10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre,15,15,15,15,15,15


## Checking unique catagory

In [48]:
len(toronto_venues['Venue Category'].unique())

269

## Lets analyze each neighborhood

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

temp_col = list(toronto_onehot.columns)
temp_col.remove('Neighborhood')
columns = ['Neighborhood'] + temp_col
toronto_onehot = toronto_onehot[columns]
toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_onehot.shape

(2133, 269)

## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
toronto_onehot_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

## Print each Neighborhood with top 5 venues

In [52]:
num_top_venues = 5

for hood in toronto_onehot_grouped['Neighborhood']:
    print("***** {} *****".format(hood))
    temp = toronto_onehot_grouped[toronto_onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

***** Agincourt *****
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2             Clothing Store  0.25
3  Latin American Restaurant  0.25
4          Accessories Store  0.00
***** Alderwood , Long Branch *****
          venue  freq
0   Pizza Place   0.2
1      Pharmacy   0.1
2           Gym   0.1
3  Skating Rink   0.1
4          Pool   0.1
***** Bathurst Manor , Wilson Heights , Downsview North *****
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2    Bridal Shop  0.05
3  Deli / Bodega  0.05
4    Supermarket  0.05
***** Bayview Village *****
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00
***** Bedford Park , Lawrence Manor East *****
                venue  freq
0  Italian Restaurant  0.09
1      Sandwich Place  0.09
2          Restaurant  0.09
3         Coffee Shop  0.09
4   Indian 

                           venue  freq
0  Paper / Office Supplies Store   0.5
1                 Baseball Field   0.5
2              Accessories Store   0.0
3      Middle Eastern Restaurant   0.0
4            Monument / Landmark   0.0
***** Humewood-Cedarvale *****
                           venue  freq
0                   Tennis Court  0.25
1                   Hockey Arena  0.25
2                          Trail  0.25
3                          Field  0.25
4  Paper / Office Supplies Store  0.00
***** India Bazaar , The Beaches West *****
                  venue  freq
0                  Park  0.10
1  Fast Food Restaurant  0.10
2     Fish & Chips Shop  0.05
3    Italian Restaurant  0.05
4           Pizza Place  0.05
***** Kennedy Park , Ionview , East Birchmount Park *****
                venue  freq
0    Department Store  0.14
1      Discount Store  0.14
2          Hobby Shop  0.14
3  Chinese Restaurant  0.14
4         Coffee Shop  0.14
***** Kensington Market , Chinatown , Grange Park *

                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.04
2   Italian Restaurant  0.03
3  Japanese Restaurant  0.03
4                Hotel  0.03
***** Studio District *****
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2               Bakery  0.05
3              Brewery  0.05
4  American Restaurant  0.05
***** Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park *****
         venue  freq
0  Coffee Shop  0.12
1          Pub  0.12
2  Pizza Place  0.06
3   Restaurant  0.06
4         Bank  0.06
***** The Annex , North Midtown , Yorkville *****
               venue  freq
0     Sandwich Place  0.13
1               Café  0.13
2        Coffee Shop  0.09
3           Pharmacy  0.04
4  Indian Restaurant  0.04
***** The Beaches *****
                venue  freq
0   Health Food Store  0.25
1               Trail  0.25
2                 Pub  0.25
3   Accessories Store  0.00
4  Miscellaneous Shop  0.00
***** The Da

## Function to return top venues

In [53]:
def return_most_common_venues(row, num_top_venue):
    temp = row.iloc[1:]
    temp_sorted = temp.sort_values(ascending=False)
    return temp_sorted.index.values[0:num_top_venue]

## create the new dataframe and display the top 10 venues for each neighborhood.

In [54]:
return_most_common_venues(toronto_onehot_grouped.iloc[0,:],10)

array(['Lounge', 'Latin American Restaurant', 'Breakfast Spot',
       'Clothing Store', 'Doner Restaurant', 'Dim Sum Restaurant',
       'Diner', 'Discount Store', 'Distribution Center', 'Dog Run'],
      dtype=object)

In [55]:
num_venues = 10
indicator = ['st','nd','rd']

columns=['Neighborhood']
for i in np.arange(num_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicator[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

toronto_sorted = pd.DataFrame(columns=columns)
toronto_sorted['Neighborhood'] = toronto_onehot_grouped['Neighborhood']

for ind in np.arange(toronto_onehot_grouped.shape[0]):
    toronto_sorted.iloc[ind,1:] = return_most_common_venues(toronto_onehot_grouped.iloc[ind,:],num_venues)

In [56]:
toronto_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood , Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports,Pool,Skating Rink,Coffee Shop,Gym,Colombian Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Supermarket,Pharmacy,Restaurant,Fried Chicken Joint,Ice Cream Shop,Diner,Middle Eastern Restaurant,Pizza Place
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Juice Bar,Hobby Shop,Thai Restaurant,Sushi Restaurant,Pizza Place,Pharmacy
5,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Café,Restaurant,Beer Bar,Cheese Shop,Farmers Market,Beach
6,"Birch Cliff , Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
7,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Stadium,Italian Restaurant,Intersection,Bar
8,Business reply mail Processing Centre,Light Rail Station,Garden,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
9,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Boutique,Coffee Shop,Airport,Airport Food Court,Airport Gate,Airport Terminal,Sculpture Garden,Plane


 ## Cluster Neighborhoods

In [57]:
ncluster=5

toronto_group_clustring = toronto_onehot_grouped.drop('Neighborhood',axis=1)
kmeans = KMeans(n_clusters=ncluster, random_state=0).fit(toronto_group_clustring)

In [58]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 4, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0], dtype=int32)

In [59]:
toronto_group_clustring.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
toronto_sorted.insert(0,'Cluster Label',kmeans.labels_)

In [61]:
toronto_sorted.head()

,Cluster Label,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,1,"Alderwood , Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports,Pool,Skating Rink,Coffee Shop,Gym,Colombian Restaurant
2,1,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Supermarket,Pharmacy,Restaurant,Fried Chicken Joint,Ice Cream Shop,Diner,Middle Eastern Restaurant,Pizza Place
3,1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,1,"Bedford Park , Lawrence Manor East",Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Juice Bar,Hobby Shop,Thai Restaurant,Sushi Restaurant,Pizza Place,Pharmacy


In [62]:
toronto_merged = toronto_merge
toronto_merged = toronto_merged.merge(toronto_sorted,on='Neighborhood')

In [63]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,French Restaurant,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Café,Breakfast Spot,Performing Arts Venue,Farmers Market,Playground
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Boutique,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Bar,Beer Bar,Sandwich Place,Burrito Place,Café,Park


## Add the lables to the Map

In [64]:
toronto_map = folium.Map(location=[latitude,longitude],zoom_start=11)

x = np.arange(ncluster)
ys = [i + x + (i*x)**2 for i in range(ncluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat,lng,poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    popup=label,
    radius=5,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_capacity=0.7).add_to(toronto_map)
toronto_map

## Examine the cluster

In [65]:
toronto_merged[toronto_merged['Cluster Label'] == 0].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
19,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Women's Store,Pool,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
33,M4J,East York,East Toronto,43.685347,-79.338106,0,Convenience Store,Park,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
49,M6L,North York,"North Park , Maple Leaf Park , Upwood Park",43.713756,-79.490074,0,Park,Bakery,Construction & Landscaping,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
59,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Falafel Restaurant


In [66]:
toronto_merged[toronto_merged['Cluster Label'] == 1].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,French Restaurant,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Café,Breakfast Spot,Performing Arts Venue,Farmers Market,Playground
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Boutique,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Bar,Beer Bar,Sandwich Place,Burrito Place,Café,Park
5,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [67]:
toronto_merged[toronto_merged['Cluster Label'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M9L,North York,Humber Summit,43.756303,-79.565963,2,Pizza Place,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [68]:
toronto_merged[toronto_merged['Cluster Label'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Garden,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega


In [69]:
toronto_merged[toronto_merged['Cluster Label'] == 4]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,4,Hockey Arena,Tennis Court,Field,Trail,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
30,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Convenience Store,Playground,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
79,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160,4,Gym,Playground,Trail,Tennis Court,Comfort Food Restaurant,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore


# Assign a Name to the Cluster
### As per my analysis 
### 0 --> Garden
### 1 -->  Food
### 2 -->  Electronics
### 3 --> Yoga
### 4 --> Venues

In [70]:
toronto_final = toronto_merged.copy()


In [71]:
toronto_final.loc[toronto_final['Cluster Label'] == 0, 'Cluster Label'] = 'Garden'
toronto_final.loc[toronto_final['Cluster Label'] == 1, 'Cluster Label'] = 'Food'
toronto_final.loc[toronto_final['Cluster Label'] == 2, 'Cluster Label'] = 'Electronics'
toronto_final.loc[toronto_final['Cluster Label'] == 3, 'Cluster Label'] = 'Yoga'
toronto_final.loc[toronto_final['Cluster Label'] == 4, 'Cluster Label'] = 'Venues'

In [72]:
toronto_final[['Borough','Neighborhood','Cluster Label']].groupby('Cluster Label').count()

,Borough,Neighborhood
Cluster Label,,
Electronics,1,1
Food,83,83
Garden,10,10
Venues,3,3
Yoga,1,1
